In [28]:
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

In [29]:
df = pd.read_csv(r"..\new_datasets\datos_gramineas.csv")
df = df.set_index("fecha")
df.index = pd.to_datetime(df.index)
df = df.sort_index(ascending=True)

In [30]:
model = xgb.Booster()
model.load_model('modelo_t+1.json')

features_que_pide_el_modelo = model.feature_names

In [31]:
ultima_fila = df.tail(6)

input_data = ultima_fila[features_que_pide_el_modelo]

In [32]:
dmatrix_input = xgb.DMatrix(input_data)

prediccion_raw = model.predict(dmatrix_input)[0]

print(f"La predicción de polen para el próximo día es: {prediccion_raw:.2f}")

La predicción de polen para el próximo día es: 2.34
